In [1]:
## Airbnb Bookings Data Analysis
#Exploring listing and bookings data to uncover factors that drive cancellations, revenue potential, and guest behavior.

## 1. Objective
#Initial EDA and vizualisations to identify the drivers of cancellations, revenue and guest behavior

## 2. Data Overview
#This data is publicly available on kaggle.com and contains

## 3. Cleaning & Preparation
#Missing data, formatting, value sanity checks.

## 4. Key Questions Explored
# What listing features correlate with high occupancy?
# Does cancellation policy impact pricing or reviews?
# How does location affect guest behavior?

## 5. Visual Insights
#Charts, grouped metrics, comparisons.

## 6. Observations & Takeaways
#Summary of business-relevant insights.


In [2]:
# importing necessary libraries
import pandas as pd
import numpy as np
import time
import glob
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy import stats
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples

# dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
#import umap

# clustering
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.mixture import GaussianMixture

In [3]:
## Part 1: Importing, Cleaning & EDA

### Importing

In [4]:
#orginal df
df=pd.read_csv(r'C:\Users\riesn\Documents\vacation-rental-bookings-analysis\data\Airbnb_Open_Data.csv', delimiter=',', encoding='utf-8', low_memory=False)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   host id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  object 
 4   host name                       102193 non-null  object 
 5   neighbourhood group             102570 non-null  object 
 6   neighbourhood                   102583 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102067 non-null  object 
 10  country code                    102468 non-null  object 
 11  instant_bookable                102494 non-null  object 
 12  cancellation_pol

In [6]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,country code,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,US,False,strict,Private room,2020.0,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,US,False,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,US,True,flexible,Private room,2005.0,$620,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,US,True,moderate,Entire home/apt,2005.0,$368,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,US,False,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [7]:
# Cleaning price and service fee columns
def clean_currency(x):
    # Remove any non-numeric characters such as $ or commas and convert to float
    if pd.isnull(x):
        return np.nan
    try:
        x = str(x).replace('$', '').replace(',', '').strip()
        return float(x)
    except Exception as e:
        # In case conversion fails, log the incident and return NaN
        # This method resolves common errors when numeric conversions are attempted on improperly formatted strings.
        return np.nan

# Apply cleaning function to relevant columns
for col in ['price', 'service fee']:
    if col in df.columns:
        df[col] = df[col].apply(clean_currency)

# Handle missing values in reviews per month by filling with 0 (i.e., no reviews per month)
if 'reviews per month' in df.columns:
    df['reviews per month'] = df['reviews per month'].fillna(0)

# Check the cleaned columns
print('Data types after cleaning:')
print(df[['price', 'service fee', 'reviews per month']].dtypes)

# Optional: Drop or fill other missing values as needed
# For this analysis we proceed without dropping rows, but note that this may be adjusted in future analysis.

Data types after cleaning:
price                float64
service fee          float64
reviews per month    float64
dtype: object


In [8]:
#going to clean up the column names for ease
orig_cols = list(df.columns)
new_cols = []
for col in orig_cols:
    new_cols.append(col.strip().replace('  ', ' ').replace(' ', '_').lower())
df.columns = new_cols

In [9]:
## Exploring Categorical variables

In [11]:
def print_unique_values_with_counts(df, columns):
    for col in columns:
        print(f"\n📌 {col} — Unique Values, Counts, and Percentages:")
        value_counts = df[col].value_counts(dropna=False)
        percentages = df[col].value_counts(normalize=True, dropna=False) * 100
        summary = pd.DataFrame({'Count': value_counts, 'Percentage': percentages.round(2)})
        print(summary)
        print('-' * 60)

# Example usage:
columns_to_check = ['neighbourhood','neighbourhood_group','country', 'country_code', 'instant_bookable', 'cancellation_policy', 'room_type']
print_unique_values_with_counts(df, columns_to_check)


📌 neighbourhood — Unique Values, Counts, and Percentages:
                            Count  Percentage
neighbourhood                                
Bedford-Stuyvesant           7937        7.74
Williamsburg                 7775        7.58
Harlem                       5466        5.33
Bushwick                     4982        4.86
Hell's Kitchen               3965        3.86
Upper West Side              3860        3.76
Upper East Side              3680        3.59
East Village                 3492        3.40
Midtown                      3390        3.30
Crown Heights                3262        3.18
East Harlem                  2341        2.28
Chelsea                      2285        2.23
Greenpoint                   2143        2.09
Lower East Side              1948        1.90
Astoria                      1876        1.83
Washington Heights           1779        1.73
Financial District           1490        1.45
West Village                 1484        1.45
Flatbush             

In [12]:
df['neighbourhood_group'] = df['neighbourhood_group'].str.strip().str.title()
replacements = {
    "Brookln": "Brooklyn",
    "Manhatan": "Manhattan"
}

df['neighbourhood_group'] = df['neighbourhood_group'].replace(replacements)

In [13]:
df_cleaned = df.dropna(subset=['neighbourhood_group']).reset_index(drop=True)

In [14]:
df_cleaned.head()

,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,country,country_code,instant_bookable,cancellation_policy,room_type,construction_year,price,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,US,False,strict,Private room,2020.0,966.0,193.0,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,US,False,moderate,Entire home/apt,2007.0,142.0,28.0,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,US,True,flexible,Private room,2005.0,620.0,124.0,3.0,0.0,NaN,0.00,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,US,True,moderate,Entire home/apt,2005.0,368.0,74.0,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,US,False,moderate,Entire home/apt,2009.0,204.0,41.0,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [15]:
#exploring continous variables:
def  describe_continuous_variables(df):
    # Select only numeric columns (int and float)
    numeric_df = df.select_dtypes(include=['number'])
    
    # Return the descriptive statistics
    return numeric_df.describe()

summary = describe_continuous_variables(df)
print(summary)

                 id       host_id            lat           long  \
count  1.025990e+05  1.025990e+05  102591.000000  102591.000000   
mean   2.914623e+07  4.925411e+10      40.728094     -73.949644   
std    1.625751e+07  2.853900e+10       0.055857       0.049521   
min    1.001254e+06  1.236005e+08      40.499790     -74.249840   
25%    1.508581e+07  2.458333e+10      40.688740     -73.982580   
50%    2.913660e+07  4.911774e+10      40.722290     -73.954440   
75%    4.320120e+07  7.399650e+10      40.762760     -73.932350   
max    5.736742e+07  9.876313e+10      40.916970     -73.705220   

       construction_year          price    service_fee  minimum_nights  \
count      102385.000000  102352.000000  102326.000000   102190.000000   
mean         2012.487464     625.293536     125.026924        8.135845   
std             5.765556     331.671614      66.325739       30.553781   
min          2003.000000      50.000000      10.000000    -1223.000000   
25%          2007.000000  

In [17]:
# in the continuous variables there seems to be some interesting outliers:
#-max 3677 days available over then next 365?
#- max calculated host listing count at 332?
#... etc 

In [20]:
def flag_outliers_all_numeric(df, suffix="_outlier"):
    """
    Flags outliers across all numeric columns in a DataFrame using the IQR method.
    Adds a new boolean column for each original numeric column.
    """
    df_out = df.copy()
    numeric_cols = df_out.select_dtypes(include='number').columns

    for col in numeric_cols:
        Q1 = df_out[col].quantile(0.25)
        Q3 = df_out[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        outlier_col = f"{col}{suffix}"
        df_out[outlier_col] = df_out[col].apply(
            lambda x: True if x < lower or x > upper else False if pd.notnull(x) else None
        )

    return df_out

In [21]:
df_flagged = flag_outliers_all_numeric(df)

# See which rows are outliers in 'price'
df_flagged[df_flagged['price_outlier'] == True]

# Count of outliers per column
df_flagged.filter(like='_outlier').sum()

id_outlier                                    0
host_id_outlier                               0
lat_outlier                                 916
long_outlier                               6539
construction_year_outlier                     0
price_outlier                                 0
service_fee_outlier                           0
minimum_nights_outlier                    18394
number_of_reviews_outlier                 11736
reviews_per_month_outlier                  5747
review_rate_number_outlier                    0
calculated_host_listings_count_outlier    17829
availability_365_outlier                      1
dtype: object